In [2]:
#!/usr/bin/env python3

import sys
import os
import glob
import itertools

import pandas as pd
from collections import Counter, defaultdict, OrderedDict
from string import ascii_lowercase

from Bio import AlignIO, SeqIO
from Bio.Alphabet import IUPAC
from Bio.Seq import MutableSeq, translate

from ind import findEnds

In [9]:
def find_dna_subs(read, ref, verbose=False, start_offset=0, end_trail=0):
    """
    @ read, ref: MutableSeq objects
    :return errors - tuple (position, expected triplet, actual triplet, ) / none if broken read
    
    This is intended purely for counting PCR polymerase substitutions.
    The assumption is that the reference includes 0 nt either side of the gene of interest.
    As for HGVS, the starting offset and number of trailing nt are variable
    Letter by letter report mutations in NGS read, all counts 1- based in result (code in 0-count).
    - substitution: 78C = nt 78 in reference is changed to C
    - deletions: 78d6 = 6 nt deleted starting with 78: 1-77, d6, 84-end
    - insertion: 78iATC = after nt 78 inserted seq. ATC
    """

    if read is None:
        if verbose:
            print('no read provided')
        return

#     # find the ends of the read
    ends = findEnds(read, ref, start_offset)

    # scan read & reference letter by letter, counting position in reference
    # reads have been trimmed so that reference starts @ offset=3 by default (0,1,2 is the extra triplet)
    dna_errors = []
    ref_index = ends.get('start') - start_offset
    i = ends.get('start')
    max_i = len(ref) - end_trail
    
    # check for indels
    if str(read).find('-') != -1:
        if verbose:
            print('indel')
        return
    elif str(ref).find('-') != -1:
        if verbose:
            print('indel')
        return
    elif len(ref) != len(read):
        if verbose:
            print('indel')
        return
    else:
        # continue on to scand the read for substitutions
        while i < ends.get('end'):
            if i > max_i:
                break
            # check for differences
            if read[i] == ref[i]:
                ref_index += 1
                i += 1

            else:
                # substitution
                if ref_index > 0:
                    dna_errors += [(str(ref_index + 1), 's', str(read[i]))]
                i += 1
                ref_index += 1


    return tuple(dna_errors)

In [ ]:
test_ref = "ATATG"
test_seq = "ATTTG"

d = find_dna_subs(test_seq, test_ref)
print(d)

In [3]:
os.chdir("../input_fasta")

In [ ]:
print(wt_fw)

In [ ]:
len(wt_fw)

The fixed length of sequences means that pre-aligning the sequences is not necessary. I can directly compare each FASTA read with the reference.

In [34]:
# let's analyse to all forward sequences
with open("./TEV_fw.fa", "r") as f:
    ref_seq =  SeqIO.read(f, "fasta", alphabet=IUPAC.ambiguous_dna)

files = glob.glob("/home/mp/InDelScanner/input_fasta/*_fw*")

dna_variant_counts = {}

for f in files:
    # make a dictionary to collect the counts
    fname = f[34:]
    dna_variant_counts[fname] = defaultdict(int)
    
    for var_record in SeqIO.parse(f, "fasta", alphabet=IUPAC.ambiguous_dna):
        dna_d = find_dna_subs(var_record.seq, ref_seq.seq, start_offset = 0, end_trail = 0)
        if dna_d is None:
            print(var_record)
        else:
            dna_variant_counts[fname][len(dna_d)] += 1

In [35]:
max_mutations  = max([max(dna_variant_counts[i].keys()) for i in dna_variant_counts.keys()])

In [36]:
column_order = ['H_fw_1.fa', 'H_fw_3.fa', 'H_fw_5.fa', 
                'H_fw_7.fa', 'H_fw_9.fa', 'H_fw_11.fa', 
                'L_fw_1.fa','L_fw_3.fa', 'L_fw_5.fa', 
                'L_fw_7.fa', 'L_fw_9.fa', 'L_fw_11.fa']

In [37]:
df = OrderedDict()

for fname in column_order:
    df[fname] = [dna_variant_counts[fname][i] for i in range(max_mutations +1)]
    
pd.DataFrame.from_dict(df)

,H_fw_1.fa,H_fw_3.fa,H_fw_5.fa,H_fw_7.fa,H_fw_9.fa,H_fw_11.fa,L_fw_1.fa,L_fw_3.fa,L_fw_5.fa,L_fw_7.fa,L_fw_9.fa,L_fw_11.fa
0,18,0,0,0,0,0,41,17,10,0,0,2
1,23,4,0,0,1,0,36,25,13,0,0,1
2,24,9,0,0,3,0,7,20,16,1,0,0
3,13,20,4,2,2,0,4,14,19,3,8,4
4,3,23,3,0,3,0,1,10,10,16,23,6
5,5,15,11,7,4,2,0,0,14,27,19,9
6,2,8,10,8,12,3,1,1,5,15,13,25
7,0,7,13,11,12,7,0,1,1,16,12,14
8,0,2,17,16,13,10,0,1,1,10,4,9
9,0,3,12,17,10,14,0,0,0,5,5,13


In [38]:
# now let's analyse to all second half sequences
with open("./TEV_rv.fa", "r") as f:
    wt_end =  SeqIO.read(f, "fasta", alphabet=IUPAC.ambiguous_dna)

dna_variant_counts = {}

files = glob.glob("/home/mp/InDelScanner/input_fasta/*_rv*")

for f in files:
    # make a dictionary to collect the counts
    fname = f[34:]
    dna_variant_counts[fname] = defaultdict(int)
    
    for var_record in SeqIO.parse(f, "fasta", alphabet=IUPAC.ambiguous_dna):
        dna_d = find_dna_subs(var_record.seq, wt_end.seq, start_offset = 0, end_trail = 0)
        if dna_d is None:
            print(var.record)
        else:
            dna_variant_counts[fname][len(dna_d)] += 1

observed_counts = [list(dna_variant_counts[i].keys()) for i in dna_variant_counts.keys()]
max_mutations = max(list(itertools.chain.from_iterable(observed_counts))) #  this tolerates an empty input file

In [41]:
dna_variant_counts.keys()

dict_keys(['H_rv_1.fa', 'H_rv_11.fa', 'H_rv_3.fa', 'H_rv_5.fa', 'H_rv_7.fa', 'H_rv_9.fa', 'L_rv_1.fa', 'L_rv_11.fa', 'L_rv_3.fa', 'L_rv_5.fa', 'L_rv_7.fa', 'L_rv_9.fa', 'TEV_rv.fa'])

In [42]:
column_order = ['H_rv_1.fa', 'H_rv_3.fa','H_rv_5.fa',
                'H_rv_7.fa', 'H_rv_9.fa','H_rv_11.fa',
                'L_rv_1.fa', 'L_rv_3.fa', 'L_rv_5.fa',
                'L_rv_7.fa', 'L_rv_9.fa', 'L_rv_11.fa']

In [53]:
df = OrderedDict()

for fname in column_order:
    df[fname] = [dna_variant_counts[fname][i] for i in range(max_mutations +1)]
    
pd.DataFrame.from_dict(df)

,H_rv_1.fa,H_rv_3.fa,H_rv_5.fa,H_rv_7.fa,H_rv_9.fa,H_rv_11.fa,L_rv_1.fa,L_rv_3.fa,L_rv_5.fa,L_rv_7.fa,L_rv_9.fa,L_rv_11.fa
0,14,0,0,0,0,0,48,13,11,0,0,0
1,25,4,0,0,0,0,47,28,17,1,0,0
2,27,4,0,0,0,0,8,21,26,1,0,0
3,11,7,0,0,0,0,7,15,17,5,1,0
4,8,12,2,0,3,0,0,12,14,9,1,0
5,7,18,2,0,1,0,1,4,4,19,1,1
6,0,12,6,0,2,3,0,0,5,27,2,0
7,0,13,9,0,5,2,0,0,1,17,11,11
8,0,11,15,0,5,8,0,0,0,9,16,24
9,0,8,20,0,8,5,0,0,0,6,13,23


### Protein sequence variation

-	Forward files contain AA 2-121
-	Reverse files contain AA 122-236, also in forward read orientation; thus naming the files as reverse is misleading.


In [21]:
# test protein_diff function for numbering

test_ref =  "AAGAGCTTGTTTAAGGGGCCGCGTGATTACAACCCGATATCGAGCACCATTTGTCATTTGACGAATGAATCTGATGGGCACACAACATCGTTGTATGGTATTGGATTTGGTCCCTTCATCATTACAAACAAGCACTTGTTTAGAAGAAATAATGGAACACTGTTGGTCCAATCACTACATGGTGTATTCAAGGTCAAGAACACCACGACTTTGCAACAACACCTCATTGATGGGAGGGACATGATAATTATTCGCATGCCTAAGGATTTCCCACCATTTCCTCAAAAGCTGAAATTTAGAGAGCCACAAAGGGAAGAGCGCATATGTCTTGTGACAACCAACTTCCAAACTAAGAGCATG"
test_read = "AAGAGCTTGTTTAAGGGGCCGCGTGATTACAACCCGATATCGAGCACCATTTGTCATTTGACGAATGAATCTGATGGGCACACAACATCGCTGTATGGTATTGGATTTGGTCCCTTCGTCATTACAAACAAGCACTTGTTTAGAAGAAATAATGGAACACTGTTGGTCCAATCACTACATGGTGTATTCAAGGTCAAGAACACCACGACTTTGCAACAACACCTCATTGATGGGAGGGACATGATAATTATTCGCATGCCTAAGGATTTCCCACCATTTCCTCAAAAGCTGACATTTAGAGAGCCACAAAGGGAAGAGCGCCTATGTCTTGTGACAACCAACTTCCAAACTAAGAGCAGG"

print(find_dna_subs(test_read, test_ref))
find_protein_diff(test_read, test_ref, start_offset = -3, end_trail = 0)

# since first AA should be position 2, use start_offset -3
# I need to disable end matching from the find_protein_diff function

(('91', 's', 'C'), ('118', 's', 'G'), ('293', 's', 'C'), ('322', 's', 'C'), ('359', 's', 'G'))


(((41, 's', 'V'), (99, 's', 'T'), (109, 's', 'L'), (121, 's', 'R')),
 'I41V/K99T/I109L/M121R')

In [20]:
def find_protein_diff(read, ref, verbose=False, start_offset=3, end_trail=3):

    # quality control
    if read is None:
        return None, None
    ends = findEnds(read, ref, start_offset)
#     if not endMatch(read, ref, ends):
#         return None, None

    newread = read
    newref = ref

    # scan reference triplet by triplet
    # move letters when encountering an indel
    prot_errors = []
    prot_short = []

    i = ends.get('aligned')
    ref_index = int((ends.get('aligned') - start_offset)/3) + 1  # reference amino acid index
    max_i = len(ref) - end_trail

    while i <= ends.get('end'):
        if i > max_i:
            break

        if newread is None:
            break
        ref_codon = newref[i:i+3]
        read_codon = newread[i:i+3]

        if '-' in read_codon:  # found a deletion
            # Check if this is the last acid, and it's incomplete, ignore it.
            if re.search('[ATGC]', str(newread[i + 3:])) is None:
                break

            if '-' in ref_codon:  # something very broken
                prot_errors.append((ref_index, 'f'))
                prot_short.append('f')
                break
            elif read_codon == '---':  # single codon deletion
                if ref_index > 0:
                    prot_errors += [(ref_index, 'd')]
                    prot_short.append(str(ref_index) + 'Δ')
                i += 3
                ref_index += 1

            else:  # check it's not a frame shift
                l = indel_len(newread, i)
                if l % 3 != 0:
                    prot_errors.append((ref_index, 'f'))
                    prot_short.append('f')
                    break
                # realign gap and repeat loop at same position to compare the codons
                gap = findGap(newread[i - 1:])
                gap = (gap[0] + i - 1, gap[1] + i - 1)
                newread = gapAlign(newread, gap, start_offset)
                continue

        elif '-' in ref_codon:  # found an insertion
            l = indel_len(newref, i)
            if l % 3 != 0:
                prot_errors.append((ref_index, 'f'))
                prot_short.append('f')
                break
            gap = findGap(newref[i-1:])
            if gap[0] == 1:  # insertion after codon
                insertion = newread[gap[0] + i - 1:gap[1] + i - 1]
                if '-' in insertion:
                    prot_errors.append((ref_index, 'f'))
                    prot_short.append('f')
                    break
                if ref_index > 0:
                    prot_errors.append((ref_index - 1, 'i', str(translate(insertion)))) # position before + insertion
                    stop, inslist = format_insertion(ref_index - 1, insertion)
                    prot_short += inslist
                    if stop:
                        break
                i += l
                ref_index += 1
            else:  # realign gap and repeat loop at same position to compare the codons
                gap = (gap[0] + i - 1, gap[1] + i - 1)
                newref = gapAlign(newref, gap, start_offset)
                continue

        elif translate(read_codon) != translate(ref_codon):  # must be a substitution
            if ref_index > 0:
                prot_errors.append((ref_index, 's', str(translate(read_codon))))
                prot_short.append(str(translate(ref_codon) + str(ref_index) + str(translate(read_codon))))
            if str(translate(read_codon)) == '*':
                break
            i += 3
            ref_index += 1

        else:
            i += 3
            ref_index += 1

    if verbose:
        print(prot_errors)

    if prot_short == []:
        short = 'wt'
    else:
        short = '/'.join(prot_short)

    return tuple(prot_errors), short


In [54]:
# now let's analyse to all second half sequences
with open("./TEV_rv.fa", "r") as f:
    ref_seq =  SeqIO.read(f, "fasta", alphabet=IUPAC.ambiguous_dna)

protein_variant_counts = {}

files = glob.glob("/home/mp/InDelScanner/input_fasta/*_rv*")

for f in files:
    # make a dictionary to collect the counts
    fname = f[34:]
    protein_variant_counts[fname] = defaultdict(int)
    
    for var_record in SeqIO.parse(f, "fasta", alphabet=IUPAC.ambiguous_dna):
        dna_d = find_dna_subs(var_record.seq, ref_seq.seq, start_offset = 0, end_trail = 0)
        if dna_d is None:
            print(var.record)
        else:
            protein_d, prot_short = find_protein_diff(var_record.seq, ref_seq.seq, start_offset = -3, end_trail = 0)
            protein_variant_counts[fname][len(protein_d)] += 1

In [31]:
dna_variant_counts.keys()

NameError: name 'dna_variant_counts' is not defined

In [30]:
protein_variant_counts.keys()

dict_keys(['H_rv_1.fa', 'H_rv_11.fa', 'H_rv_3.fa', 'H_rv_5.fa', 'H_rv_7.fa', 'H_rv_9.fa', 'L_rv_1.fa', 'L_rv_11.fa', 'L_rv_3.fa', 'L_rv_5.fa', 'L_rv_7.fa', 'L_rv_9.fa', 'TEV_rv.fa'])

In [26]:
max_mutations = max(list(itertools.chain.from_iterable(protein_variant_counts))) #  this tolerates an empty input file

In [29]:
list(itertools.chain.from_iterable(protein_variant_counts))

['H',
 '_',
 'r',
 'v',
 '_',
 '1',
 '.',
 'f',
 'a',
 'H',
 '_',
 'r',
 'v',
 '_',
 '1',
 '1',
 '.',
 'f',
 'a',
 'H',
 '_',
 'r',
 'v',
 '_',
 '3',
 '.',
 'f',
 'a',
 'H',
 '_',
 'r',
 'v',
 '_',
 '5',
 '.',
 'f',
 'a',
 'H',
 '_',
 'r',
 'v',
 '_',
 '7',
 '.',
 'f',
 'a',
 'H',
 '_',
 'r',
 'v',
 '_',
 '9',
 '.',
 'f',
 'a',
 'L',
 '_',
 'r',
 'v',
 '_',
 '1',
 '.',
 'f',
 'a',
 'L',
 '_',
 'r',
 'v',
 '_',
 '1',
 '1',
 '.',
 'f',
 'a',
 'L',
 '_',
 'r',
 'v',
 '_',
 '3',
 '.',
 'f',
 'a',
 'L',
 '_',
 'r',
 'v',
 '_',
 '5',
 '.',
 'f',
 'a',
 'L',
 '_',
 'r',
 'v',
 '_',
 '7',
 '.',
 'f',
 'a',
 'L',
 '_',
 'r',
 'v',
 '_',
 '9',
 '.',
 'f',
 'a',
 'T',
 'E',
 'V',
 '_',
 'r',
 'v',
 '.',
 'f',
 'a']

In [ ]:
dna_errors = find_dna_diff(read, ref, debug, start_offset, end_trail)  # errors = a tuple
dna_hgvs = find_dna_hgvs(read, ref, refname, debug, start_offset, end_trail)  # string in HGVS format (ish)
prot_errors, prot_short = find_protein_diff(read, ref, debug, start_offset, end_trail)